In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'
#os.environ["CUDA_VISIBLE_DEVICES"] = '4,5,7'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
from torch.optim.lr_scheduler import StepLR
import time
import customTree
import math

In [132]:
def generate_target_sim(sim, topk=5, power=2):
    target_matrix = np.zeros((1000,1000))
    for i in range(1000):
        order = sim[i].argsort()[::-1]
        min_v = sim[i][order[topk]]
        index = order[:topk+1]
        for j in range(topk,1000):
            if sim[i][order[j]]<min_v:
                gap = sim[i][order[j]]
                break
        for j in range(i+1):
            target_matrix[i,j] = pow(max(0,sim[i,j]-gap)/(1-gap), power) if j in index else 0
            target_matrix[j,i] = target_matrix[i,j]
    return target_matrix

In [12]:
a = np.array([1,1,2,3,5,4])
order = 5-np.argsort(a)
a[np.where(order == 2)[0][0]]

3

In [16]:
similarity = np.load('data/wu_palmer_similarity_matrix.npy')

In [138]:
mat = generate_target_sim(similarity,power=2)

In [139]:
np.max(np.sum(mat,axis=1))

3.3669844824051274

In [131]:
i=992
sim=similarity
topk=5

order = sim[i].argsort()[::-1]
min_v = sim[i][order[topk]]
index = order[:topk+1]
for j in range(topk,1000):
    if sim[i][order[j]]<min_v:
        print(j,order[j])
        gap = sim[i][order[j]]

7 257
8 284
9 171
10 69
11 111
12 110
13 172
14 281
15 107
16 283
17 282
18 116
19 268
20 327
21 285
22 328
23 254
24 329
25 79
26 981
27 256
28 983
29 255
30 253
31 251
32 267
33 266
34 155
35 151
36 154
37 153
38 152
39 112
40 258
41 101
42 265
43 246
44 247
45 248
46 117
47 115
48 114
49 113
50 159
51 108
52 259
53 260
54 261
55 262
56 263
57 264
58 158
59 242
60 243
61 8
62 982
63 80
64 81
65 82
66 83
67 85
68 86
69 70
70 71
71 7
72 230
73 199
74 223
75 193
76 164
77 163
78 162
79 226
80 157
81 194
82 195
83 234
84 231
85 235
86 238
87 239
88 200
89 240
90 241
91 201
92 202
93 203
94 233
95 222
96 227
97 192
98 232
99 168
100 173
101 174
102 175
103 176
104 177
105 178
106 181
107 182
108 183
109 184
110 185
111 186
112 229
113 161
114 187
115 191
116 228
117 160
118 73
119 244
120 93
121 315
122 316
123 317
124 318
125 319
126 320
127 245
128 96
129 95
130 94
131 92
132 314
133 88
134 9
135 87
136 72
137 78
138 985
139 77
140 76
141 75
142 74
143 23
144 204
145 25
146 308
147 124


In [130]:
gap

0.1111111111111111

In [128]:
sim[i][order[7]]

0.631578947368421

In [118]:
index

array([992, 997, 996, 995, 994, 993])

In [119]:
sim[992][index]

array([1.        , 0.82352941, 0.82352941, 0.82352941, 0.82352941,
       0.82352941])

In [114]:
similarity[992]

array([0.46153846, 0.46153846, 0.48      , 0.48      , 0.5       ,
       0.5       , 0.5       , 0.6       , 0.6       , 0.57142857,
       0.52173913, 0.52173913, 0.52173913, 0.52173913, 0.5       ,
       0.52173913, 0.5       , 0.52173913, 0.52173913, 0.52173913,
       0.54545455, 0.54545455, 0.54545455, 0.57142857, 0.54545455,
       0.57142857, 0.54545455, 0.54545455, 0.54545455, 0.54545455,
       0.54545455, 0.57142857, 0.57142857, 0.5       , 0.5       ,
       0.52173913, 0.52173913, 0.52173913, 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.52173913,
       0.52173913, 0.48      , 0.52173913, 0.52173913, 0.52173913,
       0.52173913, 0.52173913, 0.52173913, 0.52173913, 0.52173913,
       0.52173913, 0.5       , 0.48      , 0.5       , 0.48      ,
       0.54545455, 0.52173913, 0.48      , 0.48      , 0.63157895,
       0.6       , 0.6       , 0.57142857, 0.57142857, 0.57142

In [115]:
order = similarity[992].argsort()[::-1]
gap = similarity[992][order[5]]
index = order[:5+1]

In [109]:
index

array([19, 20, 11, 18, 17, 15])

In [107]:
similarity[19][index]

array([1.        , 0.88888889, 0.85714286, 0.85714286, 0.85714286,
       0.85714286])

In [108]:
max(0,0.85714286-gap)/(1-gap)

1.9999999989472876e-08

In [93]:
mat[19]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [84]:
order[0]

480

In [81]:
similarity[480,971]

0.1111111111111111

In [80]:
np.where(similarity[480]==np.min(similarity[480]))

(array([971]),)

In [63]:
np.argsort(similarity[480])[480]

994

In [58]:
np.argsort(similarity[480])[480],np.argsort(similarity[480])[29]

(994, 999)

In [59]:
similarity[480][480],similarity[480][29]

(1.0, 0.36363636363636365)

In [ ]:
np

In [56]:
similarity[480][30]

0.36363636363636365

In [37]:
top_k = similarity[9][np.argpartition(similarity[9],-5)[-5:-5+1]]
indexes = np.random.choice(np.where(similarity[9]>=top_k)[0], size=5)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [43]:
np.where(similarity[9]>=top_k)[0]

array([ 7,  8,  9, 23, 87, 88, 92, 93, 94, 95, 96])

In [20]:
target_matrix2 = generate_target_sim(similarity, topk=5, gap=0.85, power=3)

In [21]:
np.sum(target_matrix2, axis=1)

array([1.22921143, 1.22921143, 1.04930755, 1.04930755, 1.09209933,
       1.17283951, 1.17283951, 1.08110682, 1.08110682, 1.00443823,
       1.47891159, 1.47891159, 1.47891159, 1.47891159, 1.1195621 ,
       1.04807259, 1.02989052, 1.16179506, 1.16179506, 1.01818206,
       1.13940964, 1.008     , 1.008     , 1.02843823, 1.008     ,
       1.032     , 1.12362906, 1.12362906, 1.12362906, 1.12362906,
       1.016     , 1.0957915 , 1.0957915 , 1.26113934, 1.26113934,
       1.34722301, 1.34722301, 1.34722301, 1.01371742, 1.18381344,
       1.18381344, 1.01371742, 1.18381344, 1.18381344, 1.01371742,
       1.01371742, 1.01371742, 1.01371742, 1.01371742, 1.00010798,
       1.00010798, 1.        , 2.16730203, 2.16730203, 2.16730203,
       2.16730203, 2.16730203, 2.16730203, 2.16730203, 2.16730203,
       2.16730203, 1.04467792, 1.04467792, 1.        , 1.        ,
       1.17426205, 1.02114151, 1.19986711, 1.19986711, 1.03703704,
       1.06346186, 1.06346186, 1.43895748, 1.43895748, 1.43895

In [15]:
target_matrix = generate_target_sim(similarity, gap=0.054)
target_matrix = torch.from_numpy(target_matrix)

In [16]:
torch.sum(target_matrix)

tensor(1000.0000, dtype=torch.float64)

In [18]:
def modify_target(target_matrix, eps = 0.9):
    eps_sum = torch.reshape(1 + eps*(torch.sum(target_matrix,dim=1) - torch.max(target_matrix,dim=1)[0]),(-1,1))
    new_target_matrix = eps*target_matrix/eps_sum
    for i in range(target_matrix.shape[0]):
        new_target_matrix[i,i] = new_target_matrix[i,i]/(eps*target_matrix[i,i])
    return new_target_matrix    

In [21]:
a=torch.tensor([1]*1000, dtype=torch.float64)

In [96]:
batch = target_matrix[target]
batch

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [97]:
test = torch.tensor([[0.7,0.1,0.1,0.1],[0.2,0.6,0.1,0.1],[0.1,0.1,0.7,0.1],[0,0,0.1,0.9]])
torch.sum(test,dim=1)

tensor([1., 1., 1., 1.])

In [102]:
test.tolist()

[[0.699999988079071,
  0.10000000149011612,
  0.10000000149011612,
  0.10000000149011612],
 [0.20000000298023224,
  0.6000000238418579,
  0.10000000149011612,
  0.10000000149011612],
 [0.10000000149011612,
  0.10000000149011612,
  0.699999988079071,
  0.10000000149011612],
 [0.0, 0.0, 0.10000000149011612, 0.8999999761581421]]

In [9]:
transforms_train = transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
imagenet_data_train = torchvision.datasets.ImageFolder(root='../../../../local/scratch/datasets/ImageNet/ILSVRC2012/train', transform = transforms_train)
train_data_loader = torch.utils.data.DataLoader(imagenet_data_train, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

In [2]:
import pickle
w2v_list = pickle.load(open('data/word2vec/w2vl.pkl', 'rb'))
w2v_dict = dict(w2v_list)
words,vectors = zip(*w2v_list)

In [11]:
lc_w2v = {w.lower(): w2v_dict[w] for w in reversed(words)}

In [6]:
import json
class_dict = json.load(open('data/imagenet_class_index.json'))
nclass = len(class_dict); nclass

1000

In [9]:
classids_1k = dict(class_dict.values())
classid_lines = open('data/classids.txt', 'r').readlines()
classids = dict(l.strip().split(' ') for l in classid_lines)
len(classids)

82115

In [12]:
syn_wv = [(k, lc_w2v[v.lower()]) for k,v in classids.items()
          if v.lower() in lc_w2v]
syn_wv_1k = [(k, lc_w2v[v.lower()]) for k,v in classids_1k.items()
          if v.lower() in lc_w2v]
syn2wv = dict(syn_wv); len(syn2wv)

51633

In [ ]:
pickle.dump(syn_wv,open('data/syn_wv.pkl','wb'))

In [ ]:
pickle.load(open('data/syn_wv.pkl','wb'))